# **Modelo de Recomendación de Juegos (*item-item*)**

In [42]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Traemos los datasets necesarios:

In [45]:
df_genre = pd.read_parquet("./datasets_endpoints/games_genre_dummies.parquet")
df_genre.head(3)

,app_name,release_date,price,item_id,developer,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
1,Ironbound,2018-01-04,free,643980,Secret Level SRL,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,Real Pool 3D - Poolians,2017-07-24,free,670290,Poolians.com,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0


In [46]:
# Creamos una nueva columna 'year'
df_genre["release_date"] = pd.to_datetime(df_genre["release_date"], errors="coerce")
df_genre["year"] = pd.to_datetime(df_genre["release_date"]).dt.year
df_genre.head(4)

,app_name,release_date,price,item_id,developer,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,year
0,Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro,0,1,0,0,0,...,0,0,1,0,0,1,0,0,0,2018.0
1,Ironbound,2018-01-04,free,643980,Secret Level SRL,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,2018.0
2,Real Pool 3D - Poolians,2017-07-24,free,670290,Poolians.com,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,2017.0
3,弹炸人2222,2017-12-07,0.99,767400,彼岸领域,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,2017.0


In [47]:
df_genre["year"] = df_genre["year"].fillna(0)
df_genre["year"] = df_genre["year"].astype(int)

In [48]:
df_genre = df_genre.drop("release_date", axis=1)

In [49]:
scaler = StandardScaler()
df_genre['year'] = scaler.fit_transform(df_genre[['year']])

df_muestra = df_genre.sample(n=2000, random_state=42)

df_muestra.to_parquet('./datasets_endpoints/muestra_recomendacion.parquet', compression='snappy')

In [50]:
df_muestra[df_muestra["item_id"]== 569712]

,app_name,price,item_id,developer,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,year
8868,BERSERK - Casca Costume: Guts' Tank Top,3.49,569712,"KOEI TECMO GAMES CO., LTD.",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.11347


In [51]:
def recomendacion_juego(item_id: str):
    top = 5
    df_prueba = pd.read_parquet('./datasets_endpoints/muestra_recomendacion.parquet')
    df_prueba = df_prueba.reset_index(drop=True)

    # seleccionando características numéricas para utilizarlas en el cálculo de similitud entre elementos
    numericas_car = df_prueba.select_dtypes(include=[np.number]).columns.tolist()
    matriz_similitud = cosine_similarity(df_prueba[numericas_car].fillna(0))
    matriz_similitud = np.nan_to_num(matriz_similitud)
 
    # Verifica si el item_id dado no está presente en la columna 'item_id' del DataFrame
    if item_id not in df_prueba['item_id'].values:
        return f"Recomendaciones no encontradas: {item_id} no se encuentra en la base de datos."

    lista_apariciones_ids = df_prueba.index[df_prueba['item_id'] == item_id].tolist()

    # Verifica si la lista de índices de apariciones está vacía. Si está vacía, significa que el item_id no está en la lista de juegos
    if not lista_apariciones_ids:
        return f"Recomendaciones no encontradas: {item_id} no esta en la lista."
    lista_apariciones_ids = lista_apariciones_ids[0]

    # Puntajes de similitud y ordenamiento
    similitud_scores = list(enumerate(matriz_similitud[lista_apariciones_ids]))
    similitud_scores = sorted(similitud_scores, key=lambda x: x[1], reverse=True)

    indices_juegos_similares = [i for i, score in similitud_scores[1:top+1]]
    juegos_similares = df_prueba['app_name'].iloc[indices_juegos_similares].tolist()


    juego_nombre = df_prueba['app_name'].iloc[lista_apariciones_ids]
    recomendacion_ms = f"Recomendación de juegos por item_id {item_id} - {juego_nombre}:"
    recomendacion_final = [recomendacion_ms] + juegos_similares

    return recomendacion_final

In [52]:
test_id = 569712  
reco = recomendacion_juego(test_id)

print("Recomendación para el juego con item_id: ", test_id, "\n")
for i in reco:
    print(i, "\n")

Recomendación para el juego con item_id:  569712 

Recomendación de juegos por item_id 569712 - BERSERK - Casca Costume: Guts' Tank Top: 

DARIUSBURST Chronicle Saviours - Side Arms Hyper Dyne 

BERSERK - Exclusive Costumes 

Tom Clancy's Rainbow Six® Siege - Tachanka Bushido Set 

Tom Clancy's Rainbow Six® Siege - Smoke Bushido Set 

DOA5LR AQUAPLUS Mashup Set 

